# You have to develop a shopping application or e-commerce application which has login and public login features on the Python platform. The applications that have been developed should also include categories, such as 3–4 for footwear, clothing, electronics, etc. It should be possible to add and update categories in the application. Additionally, it must contain a feature that allows you to add or remove items from your cart. Finally, the program needs to support a variety of payment options, including UPI and debit cards. This should be only backend implementation, and UX/UI and database connectivity are not required.

In [271]:
# Initialization

# User

class User:
    def __init__(self, username, password, isAdmin=False):
        self.__username = username
        self.__password = password
        self.__isAdmin = isAdmin
        self.__cart = Cart()
        
    def login(self, username, password):
        return self.__username == username and self.__password == password

    @property
    def cart(self):
        return self.__cart

    @property
    def isAdmin(self):
        return self.__isAdmin

    @property
    def getUsername(self):
        return self.__username

In [272]:
# Category

class Category:
    def __init__(self, name):
        self.__name = name
        self.__items = []

    def addItem(self, item):
        self.__items.append(item)

    def removeItem(self, itemName):
        self.__items = [item for item in self.__items if item.name != itemName]

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, name):
        self.__name = name

    @property
    def items(self):
        return self.__items

In [273]:
# Item

class Item:
    def __init__(self, name, price, quantity):
        self.__name = name
        self.__price = price
        self.__quantity = quantity

    @property
    def name(self):
        return self.__name

    @property
    def price(self):
        return self.__price

    @property
    def quantity(self):
        return self.__quantity

    def updateQuantity(self, quantity):
        if quantity >= 0:
            self.__quantity = quantity
        else:
            raise ValueError("Quantity cannot be negative")

In [274]:
# Cart

class Cart:
    def __init__(self):
        self.__items = {}
    
    def addItem(self, item, quantity):
        if item.quantity >= quantity:
            if item.name in self.__items:
                self.__items[item.name]['quantity'] += quantity
            else:
                self.__items[item.name] = {'item': item, 'quantity': quantity}
            item.updateQuantity(item.quantity - quantity)
        else:
            print(f"Insufficient quantity for {item.name}")

    def removeItem(self, item_name, quantity):
        if item_name in self.__items:
            if self.__items[item_name]['quantity'] >= quantity:
                self.__items[item_name]['quantity'] -= quantity
                self.__items[item_name]['item'].updateQuantity(
                    self.__items[item_name]['item'].quantity + quantity)
                if self.__items[item_name]['quantity'] == 0:
                    del self.__items[item_name]
            else:
                print(f"Cannot remove {quantity} items. Only {self.__items[item_name]['quantity']} items in cart.")
        else:
            print(f"{item_name} not in cart")

    def getTotal(self):
        return sum(item['item'].price * item['quantity'] for item in self.__items.values())

    @property
    def items(self):
        return self.__items

In [275]:
# Payment

class Payment:
    @staticmethod
    def process_payment(payment_method, amount):
        if payment_method in ['UPI', 'Debit Card', 'PAYPAL']:
            print(f"Processing {payment_method} payment of amount: {amount}")
            return True
        else:
            print("Invalid payment method")
            return False

In [276]:
# Shopping App

class ShoppingApp:
    def __init__(self):
        self.__users = []
        self.__categories = []

    def registerUser(self, username, password, isAdmin=False):
        user = User(username, password, isAdmin)
        self.__users.append(user)

    def loginUser(self, username, password):
        for user in self.__users:
            if user.login(username, password):
                return user
        return None

    def addCategory(self, category_name, user):
        if user.isAdmin:
            category = Category(category_name)
            self.__categories.append(category)
        else:
            print("Permission denied: Only admins can add categories.")

    def updateCategory(self, oldCategoryName, newCategoryName, user):
        if user.isAdmin:
            for category in self.__categories:
                if category.name == oldCategoryName:
                    category.name = newCategoryName
        else:
            print("Permission denied: Only admins can update categories.")

    def addItemToCategory(self, category_name, item, user):
        if user.isAdmin:
            category = self.findCategory(category_name)
            if category:
                category.addItem(item)
            else:
                print(f"Category {category_name} not found.")
        else:
            print("Permission denied: Only admins can add items to categories.")

    def findCategory(self, category_name):
        for category in self.__categories:
            if category.name == category_name:
                return category
        return None

    def printCategories(self):
        print(f"Total number of categories: {len(self.__categories)}")
        for category in self.__categories:
            print(f"Category: {category.name}")
            for item in category.items:
                print(f"  Item: {item.name}, Price: {item.price}, Quantity: {item.quantity}")

    def printItems(self):
        total_items = 0  # Initialize total items counter

        # Count total items first
        for category in self.__categories:
            for item in category.items:
                total_items += 1

        # Print total number of items
        print(f"Total number of items in all categories: {total_items}")

        # Print items details
        for category in self.__categories:
            for item in category.items:
                print(f"Category: {category.name}, Item: {item.name}, Price: {item.price}, Quantity: {item.quantity}")
                
    @property
    def categories(self):
        return self.__categories

In [277]:
# Use ShoppingApp
app = ShoppingApp()

In [278]:
# Register Users
app.registerUser("admin", "adminpass", isAdmin=True)
app.registerUser("user1", "userpass")

In [279]:
# Login as Admin with wrong password
adminUser = app.loginUser("admin", "admin")
if adminUser:
    print("Admin logged in")
else:
    print("Invalid Login Credentials")

Invalid Login Credentials


In [280]:
# Login as Admin with wrong username
adminUser = app.loginUser("admin123", "admin")
if adminUser:
    print("Admin logged in")
else:
    print("Invalid Login Credentials")

Invalid Login Credentials


In [281]:
# Login as Admin 
adminUser = app.loginUser("admin", "adminpass")
if adminUser:
    print("Admin logged in")
else:
    print("Invalid Login Credentials")

Admin logged in


In [282]:
# Admin adds categories
if (adminUser):
    app.addCategory("Footwear", adminUser)
    app.addCategory("Clothing", adminUser)
    app.addCategory("Electronics", adminUser)

# Check Category
app.printCategories()

Total number of categories: 3
Category: Footwear
Category: Clothing
Category: Electronics


In [283]:
# Admin adds items to categories
if (adminUser):
    app.addItemToCategory("Footwear", Item("Sneakers", 50, 10), adminUser)
    app.addItemToCategory("Footwear", Item("Boots", 80, 5), adminUser)
    app.addItemToCategory("Clothing", Item("T-Shirt", 20, 50), adminUser)
    app.addItemToCategory("Clothing", Item("Jeans", 40, 30), adminUser)
    
# Check Items
app.printItems()

Total number of items in all categories: 4
Category: Footwear, Item: Sneakers, Price: 50, Quantity: 10
Category: Footwear, Item: Boots, Price: 80, Quantity: 5
Category: Clothing, Item: T-Shirt, Price: 20, Quantity: 50
Category: Clothing, Item: Jeans, Price: 40, Quantity: 30


In [284]:
# Login as Regular User with wrong password
user = app.loginUser("user1", "user1")
if user:
    print(f"Regular User {user.getUsername} logged in")
else:
    print("Invalid Login Credentials")

Invalid Login Credentials


In [285]:
# Login as Regular User with wrong username
user = app.loginUser("user123", "user1")
if user:
    print(f"Regular User {user.getUsername} logged in")
else:
    print("Invalid Login Credentials")

Invalid Login Credentials


In [286]:
# Login as Regular User 
user = app.loginUser("user1", "userpass")
if user:
    print(f"Regular User <{user.getUsername}> logged in")
else:
    print("Invalid Login Credentials")

Regular User <user1> logged in


In [287]:
# User tries to add a category (should fail)
if (user):
    app.addCategory("Toys", user)

Permission denied: Only admins can add categories.


In [288]:
# User tries to add items to category (should fail)
if (user):
    app.addItemToCategory("Electronics", Item("IPad Pro M4", 2000, 100_000), user)

Permission denied: Only admins can add items to categories.


In [289]:
# User adds items to cart
if (user):
    footwearCategory = app.findCategory("Footwear")
    sneakers = next(item for item in footwearCategory.items if item.name == "Sneakers")
    user.cart.addItem(sneakers, 2)
    tshirtCategory = app.findCategory("Clothing")
    tshirt = next(item for item in tshirtCategory.items if item.name == "T-Shirt")
    user.cart.addItem(tshirt, 3)

In [290]:
# View Cart Total
print("Cart Total:", user.cart.getTotal())

# Initialize current states as dictionaries
currentState = {}

# Capture current state after payment
for category in app.categories:
    currentState[category.name] = category.items

Cart Total: 160


In [291]:
# Remove items from cart
user.cart.removeItem("Sneakers", 1)
user.cart.removeItem("T-Shirt", 2)

In [292]:
# View Cart Total (Updated Cart)
print("Updated Cart Total:", user.cart.getTotal())

Updated Cart Total: 70


In [293]:
# Process Payment
Payment.process_payment("PAYPAL", user.cart.getTotal())

Processing PAYPAL payment of amount: 70


True

In [294]:
# Print all categories and their items (After user purchase is completed)
app.printCategories()

Total number of categories: 3
Category: Footwear
  Item: Sneakers, Price: 50, Quantity: 9
  Item: Boots, Price: 80, Quantity: 5
Category: Clothing
  Item: T-Shirt, Price: 20, Quantity: 49
  Item: Jeans, Price: 40, Quantity: 30
Category: Electronics
